In [2]:
!pip install cudf

  Using cached cudf-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for cudf
Failed to build cudf


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [34 lines of output]
      running bdist_wheel
      running build
      C:\Users\tinma\OneDrive\Escritorio\tradingbot\gpu_tradingbot\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      installing to build\bdist.win-amd64\wheel
      running install
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\tinma\AppData\Local\Temp\pip-install-_ucxq6dc\cudf_e1cef3383b204616b5601502f89f52f9\setup.py", line 18, in <module>
          setup(name=pkg,
        File "C:\Users\tinma\OneDrive\Escritorio\tradingbot\gpu_tradingbot\lib\site-packages\setuptools\__init__.py", line 87, in setup
          return distutils.core.setup(

In [7]:
!pip3 install --extra-index-url https://pypi.nvidia.com cudf-cu12


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached cudf_cu12-24.4.0.tar.gz (2.7 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
        File "C:\Users\tinma\AppData\Local\Temp\pip-build-env-ph99bfsy\overlay\Lib\site-packages\nvidia_stub\wheel.py", line 148, in download_wheel
          return download_manual(wheel_directory, distribution, version)
        File "C:\Users\tinma\AppData\Local\Temp\pip-build-env-ph99bfsy\overlay\Lib\site-packages\nvidia_stub\wheel.py", line 115, in download_manual
          raise RuntimeError(f"Didn't find wheel for {distribution} {version}")
      Traceback (most recent call last):
        File "C:\Users\tinma\AppData\Local\Temp\pip-build-env-ph99bfsy\overlay\Lib\site-packages\nvidia_stub\wheel.py", line 148, in download_wheel
          return download_manual(wheel_directory, distribution, version)
        File "C:\Users\tinma\AppData\Local\Temp\pip-build-env-ph99bfsy\overlay\Lib\site-packages\nvidia_stub\wheel.py", line 115, in down

In [1]:
################################################################################################################################################################################
# Importamos las librerías necesarias y generamos conexión con Binance para descargar la lista de pares que binance maneja.
################################################################################################################################################################################
import cudf
import cupy as cp
import os
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from binance import Client

client = Client()

##################################################
# Lista de parámetros.
##################################################
# Parámetros de interpolación.
###################################
smooth_interval = 25
smooth_exp = 3
###################################
# Parámetros de estrategia.
###################################
rango = 100
std_mult = 50
z_aprox = 0.07

##################################################################################################################################
# Descarga y limpieza de datos de X días atrás a hoy.
##################################################################################################################################
recent_data = cudf.DataFrame(client.get_historical_klines('BTCUSDT','1m','1 day ago UTC'))
recent_data = recent_data.iloc[:,:6]
recent_data.columns = ['Tiempo','Open','High','Low','Close','Volume']
recent_data['Tiempo'] = cudf.to_datetime(recent_data['Tiempo'],unit='ms')
recent_data = recent_data.set_index('Tiempo')

##################################################
# Interpolación (suavizado)  del Precio (OPEN) con cuDF y obtención de derivadas con su respectiva interpolación.
##################################################

# Suavizado utilizando filtro de Savitzky-Golay
interpolacion = recent_data['Open'].rolling(window=smooth_interval).mean().fillna(method='bfill').fillna(method='ffill')

# Calcular derivadas
primer_derivada = interpolacion.diff() / (recent_data.index.to_series().diff() / np.timedelta64(1, 's'))
segunda_derivada = primer_derivada.diff() / (recent_data.index.to_series().diff() / np.timedelta64(1, 's'))
tercera_derivada = segunda_derivada.diff() / (recent_data.index.to_series().diff() / np.timedelta64(1, 's'))

# Convertir de vuelta a DataFrame de cuDF
recent_data['Interpolacion'] = interpolacion
recent_data['Primer_Derivada'] = primer_derivada
recent_data['PD_Interpolacion'] = primer_derivada.rolling(window=smooth_interval).mean().fillna(method='bfill').fillna(method='ffill')
recent_data['Segunda_Derivada'] = segunda_derivada
recent_data['SD_Interpolacion'] = segunda_derivada.rolling(window=smooth_interval).mean().fillna(method='bfill').fillna(method='ffill')
recent_data['Tercer_Derivada'] = tercera_derivada
recent_data['TD_Interpolacion'] = tercera_derivada.rolling(window=smooth_interval).mean().fillna(method='bfill').fillna(method='ffill')

initial_date_index = recent_data[recent_data['Tiempo'] == datetime.strptime('2024-04-04 05:48:00','%Y-%m-%d %H:%M:%S')].index.item() + rango
final_date_index = recent_data[recent_data['Tiempo'] == datetime.strptime('2024-04-06 05:47:00','%Y-%m-%d %H:%M:%S')].index.item() - rango

######################################################################
# Agregamos el código necesario para encontrar los puntos exactos mínimos y máximos
######################################################################
min_max_sd = cudf.DataFrame(columns=['Derivada'])
min_max_sd['Derivada'] = recent_data.iloc[initial_date_index:final_date_index]['SD_Interpolacion'].diff()/(recent_data.index.to_series().diff() / np.timedelta64(1, 's'))
lista_min_max = min_max_sd[(min_max_sd['Derivada'] >= -z_aprox) & (min_max_sd['Derivada'] <= z_aprox)]
indices_min_max = lista_min_max.index.tolist()

max_points_idx = []
min_points_idx = []
mm_points_idx = []
pmin = 0
pmax = 0

for i in range(len(indices_min_max)):
    PMSD = indices_min_max[i]
    if recent_data['SD_Interpolacion'].iloc[PMSD] > 0:
        if abs(recent_data['SD_Interpolacion'].iloc[i]*100) > abs(cp.std(recent_data['SD_Interpolacion'])*std_mult):
            pmax = recent_data['SD_Interpolacion'].iloc[PMSD]
        for i in range(rango+1):
            if (recent_data['SD_Interpolacion'].iloc[PMSD-i] > pmax) & (abs(recent_data['SD_Interpolacion'].iloc[PMSD-i]*100) > abs(cp.std(recent_data['SD_Interpolacion'])*std_mult)):
                pmax_idx = PMSD-i
            elif (recent_data['SD_Interpolacion'].iloc[PMSD+i] > pmax) & (abs(recent_data['SD_Interpolacion'].iloc[PMSD+i]*100) > abs(cp.std(recent_data['SD_Interpolacion'])*std_mult)):
                pmax_idx = PMSD+i
        max_points_idx.append(pmax_idx)
    elif recent_data['SD_Interpolacion'].iloc[PMSD] < 0:
        if abs(recent_data['SD_Interpolacion'].iloc[i]*100) > abs(cp.std(recent_data['SD_Interpolacion'])*std_mult):
            pmin = recent_data['SD_Interpolacion'].iloc[PMSD]
        for i in range(rango+1):
            if (recent_data['SD_Interpolacion'].iloc[PMSD-i] < pmin) & (abs(recent_data['SD_Interpolacion'].iloc[PMSD-i]*80) > abs(cp.std(recent_data['SD_Interpolacion'])*std_mult)):
                pmin_idx = PMSD-i
            elif (recent_data['SD_Interpolacion'].iloc[PMSD+i] < pmin) & (abs(recent_data['SD_Interpolacion'].iloc[PMSD+i]*80) > abs(cp.std(recent_data['SD_Interpolacion'])*std_mult)):
                pmin_idx = PMSD+i
        min_points_idx.append(pmin_idx)

max_points_idx = list(set(max_points_idx))
min_points_idx = list(set(min_points_idx))
max_points_idx.sort()
min_points_idx.sort()

# Convertir los índices a arrays de NumPy
max_points_idx = cp.array(max_points_idx)
min_points_idx = cp.array(min_points_idx)

# Crear DataFrames de cuDF con los índices filtrados
max_points_df = cudf.DataFrame({'Index': max_points_idx})
min_points_df = cudf.DataFrame({'Index': min_points_idx})

# Convertir de vuelta a pandas DataFrame para la visualización
max_points_df = max_points_df.to_pandas()
min_points_df = min_points_df.to_pandas()

# Visualización
mp = recent_data['Open'].iloc[initial_date_index:final_date_index].mean()
plt.figure(figsize=(15,8))
plt.plot(recent_data['Tiempo'].iloc[initial_date_index:final_date_index], recent_data['Open'].iloc[initial_date_index:final_date_index].to_array(), color='y')
plt.axvline(datetime.strptime('2024-04-04 09:59:00','%Y-%m-%d %H:%M:%S'))
plt.scatter(recent_data['Tiempo'][initial_date_index:final_date_index].iloc[max_points_idx].to_array(), recent_data['Open'][initial_date_index:final_date_index].iloc[max_points_idx].to_array(), color='g')
plt.scatter(recent_data['Tiempo'][initial_date_index:final_date_index].iloc[min_points_idx].to_array(), recent_data['Open'][initial_date_index:final_date_index].iloc[min_points_idx].to_array(), color='r')
plt.grid(visible=True)

# Configuración de ejes y formato de fecha
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%y %H:%M'))
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=60))
plt.gcf().autofmt_xdate()
plt.grid(which='minor')
plt.xticks(rotation=75)
plt.axhline(mp)
plt.show()


ModuleNotFoundError: No module named 'cudf'